In [194]:
import requests
import re
import pandas as pd
import sys
import numpy as np


In [197]:
# Get data
'''
Input of csv with a street address, city, and state column
'''
df_addresses = pd.read_csv(r"Addresses.csv")



,RecordID,AddressLine1,Addressline2,City,State,Zipcode,Longitude,Latitude
0,447,2300 Wilson Blvd Ste 100,NaN,Arlington,VA,22201,NaN,NaN
1,770,1197 N Lee Hwy,NaN,Lexington,VA,24450,NaN,NaN
2,433,11250 Staples Mill Road,NaN,Glen Allen,VA,23060,NaN,NaN
3,470,2331 Franklin Road,NaN,Roanoke,VA,24014,NaN,NaN
4,475,24010 Partnership Boulevard,NaN,Ruther Glen,VA,22546,NaN,NaN


In [198]:
## Set API Key
api_key = 

In [199]:
def udf_remove_address_unit(address):
    
    '''
    Pre-process address by capturing address line 2 references
    Input: address as a string
    Output: processed address
    
    Remove any housing unit references from the address and ensure the last character of the address is a letter
    '''
    
    # Remove Unit Name
    address = str(address)
    address_processed = address.upper()
    address_processed = address_processed.partition("APT")[0]
    address_processed = address_processed.partition("UNIT")[0]
    address_processed = address_processed.partition("SUITE")[0]
    address_processed = address_processed.partition("STE")[0]
    
    # remove all non-alphanumeric characters at the end of the address
    address_processed = re.sub(f'[^1-9a-zA-Z]+$','',address_processed)
    
    return address_processed

In [201]:
def udf_get_google_geocoded_address (address, api_key):

    geocode_url = "https://maps.googleapis.com/maps/api/geocode/json?address={}".format(address)
    geocode_url = geocode_url + "&key={}".format(api_key)

    results = requests.get(geocode_url)
    results = results.json()
    results = results['results']
    
    if len(results) == 0:
    
        output = {
            "Address_Geocoded" : None,
            "City_Geocoded" : None,
            "County_Geocoded" : None,
            "State_Geocoded" : None,
            "ZipCode_Geocoded" : None,
            "Latitude" : None,
            "Longitude" : None
        }
        
    else:

        results = results[0]
        output = {
            "Address_Geocoded" : results.get('address_components')[0].get('long_name') + ' ' + results.get('address_components')[1].get('long_name'),
            "City_Geocoded" : results.get('address_components')[2].get('long_name'),
            "County_Geocoded" : results.get('address_components')[4].get('long_name'),
            "State_Geocoded" : results.get('address_components')[5].get('short_name'),
            "ZipCode_Geocoded" : results.get('address_components')[7].get('long_name'),
            "Latitude" : results.get('geometry').get('location')['lat'],
            "Longitude" : results.get('geometry').get('location')['lng']
        }

    output['Address_Full_PreProcessed'] = address
    
    return output


In [216]:
def udf_geocode_addresses(df, address, api_key):
    
    '''
    Given a dataframe, df, of addresses, address, 
    '''
    df_geocoded_addresses = pd.DataFrame()
    
    df[(address+'_PreProcessed')] = df[address].apply(udf_remove_address_unit)

    df[(address+'_Full_PreProcessed')] = df[(address+'_PreProcessed')] + ' ' + df['City'] + ' ' + df['State'] + ' ' + df['Zipcode']
    
    for index, row in df.iterrows():
            
        df_geocoded_addresses = df_geocoded_addresses.append(udf_get_google_geocoded_address(row[(address+'_Full_PreProcessed')],api_key),ignore_index=True)
   
    return df_geocoded_addresses